In [4]:
import os
import time
import functools
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from transformers import TFViTModel

In [5]:
DATAPATH = 'Animals_10'

In [6]:
dataset_dir_path = os.path.join(DATAPATH, 'raw-img')
print(dataset_dir_path)

Animals_10/raw-img


In [7]:
# Get the list of all files and directories
get_dir_list = os.listdir(dataset_dir_path)
print(get_dir_list)

['farfalla', 'mucca']


In [8]:
translate = {"cane": "dog", "cavallo": "horse", "elefante": "elephant", "farfalla": "butterfly", "gallina": "chicken", "gatto": "cat", "mucca": "cow", "pecora": "sheep", "ragno": "spider", "scoiattolo": "squirrel", "dog": "cane", "cavallo": "horse", "elephant" : "elefante", "butterfly": "farfalla", "chicken": "gallina", "cat": "gatto", "cow": "mucca", "spider": "ragno", "squirrel": "scoiattolo"}

In [9]:
get_idx_label={}
get_label_inx = []
for idx, label in enumerate(get_dir_list):
    get_idx_label[translate[label]] = idx
    get_label_inx.append(translate[label])
print(get_idx_label)
print(get_label_inx)

{'butterfly': 0, 'cow': 1}
['butterfly', 'cow']


In [10]:
get_all_datalist , get_all_label = [], []
for get_dirt in get_dir_list:
    get_dir_path = os.path.join(dataset_dir_path,get_dir)
    get_file_list = os.listdir(get_dir_path)
    for get_file in get_file_list:
        get_file_path = os.path.join(get_dir_path,get_file)
        get_all_datalist.append(get_file_path)
        translate_lable = translate[get_file_path.splt("/")[-2]]
        get_all_label.append(translate_lable)
print(get_all_datalist)
print(set(get_all_label))

NameError: name 'get_dir' is not defined

In [11]:
train_data, test_data, train_label, test_label = train_test_split(get_all_datalist, get_all_label,test_size=0.2,random_state=42)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [12]:
print(len(train_data),len(test_label))
print(len(test_data), len(test_label))

NameError: name 'train_data' is not defined

In [13]:
train_df = pd.DataFrame({
    'data':train_data,
    'labels':train_label
})

test_df = pd.DataFrame({'data':test_data, 'labels':test_label})

print(train_df.head())
print(test_df.head())

NameError: name 'train_data' is not defined

In [14]:
batch_size = 128
image_size = 224

In [15]:
train_idg = ImageDataGenerator( rescale= 1./255,
                                rotation_range= 0.1,
                                height_shift_range=0.1,
                                width_shift_range=0.1,
                                zoom_range=0.1,
                                horizontal_flip=True,
                                vertical_flip=False)

test_idg = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_idg.flow_from_dataframe(dataframe=train_df,
                                                directory=None,
                                                x_col="data",
                                                y_col="label",
                                                class_mode="categorical",
                                                target_size=(224,224),
                                                batch_size=batch_size)

test_generator = test_idg.flow_from_dataframe(dataframe=test_df,
                                              directory=None,
                                              x_col="data",
                                              y_col="label",
                                              class_mode="categorical",
                                              target_size=(224,224),
                                              batch_size=batch_size)

In [16]:
def plotImage(image_arr):
    fig,axes = plt.subplots(1,5,figsize=(20,20))
    axes = axes.flatten() #1
    for img, ax in zip(image_arr,axes):
    
        ax.imshow(img)
        ax.axis('off') # 눈금비활성화
    plt.tight_layout() # 서브플롯의 간격을 자동조절
    plt.show()

In [17]:
augmented_images = [ train_generator[0][0][0] for i in range(5)]
plotImage(augmented_images)

NameError: name 'train_generator' is not defined

In [ ]:
base_model = TFViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

pixel_values = tf.keras.layers.Input(shape=(224, 224, 3), dtype='float32')
permute = tf.keras.layers.Permute((3, 1, 2))(pixel_values)

# model layer
vit = base_model.vit(permute)[0]
classifier = tf.keras.layers.Dense(10, name='outputs')(vit[:, 0, :])

# model
model = tf.keras.Model(inputs=pixel_values, outputs=classifier)

# define loss
loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# define metrics
metrics=tf.keras.metrics.CategoricalAccuracy(name="accuracy")

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss=loss,
              metrics=metrics)
